In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [2]:
import tensorflow as tf
import torch
import numpy as np
import math
from torch.nn import functional as F

2022-06-17 14:51:30.663692: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
/home/ubuntu/tf-nvidia/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
tf.compat.v1.enable_eager_execution()

In [4]:
# depthwise_conv = tf.keras.layers.DepthwiseConv2D((5, 1),dilation_rate = 5, padding='SAME')

In [5]:
x = tf.random.normal(shape = (2, 5, 100))

2022-06-17 14:51:36.749928: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-06-17 14:51:36.778511: E tensorflow/stream_executor/cuda/cuda_driver.cc:282] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-06-17 14:51:36.778535: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: huseincomel-desktop
2022-06-17 14:51:36.778538: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: huseincomel-desktop
2022-06-17 14:51:36.778572: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 470.129.6
2022-06-17 14:51:36.778587: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 470.129.6
2022-06-17 14:51:36.778590: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 470.129.6
2022-06-17 14:51:36.789308: I tensorflow/

In [6]:
# tf.concat([x[..., :-1], tf.expand_dims(x[..., -1] + 1e-6, -1)], axis = -1)

In [7]:
half_channels = 100
num_bins = 10
half_channels * (num_bins * 3 - 1)

2900

In [8]:
inputs = np.random.normal(size = (2, 100, 5)).astype(np.float32)
h = np.random.normal(size = (2, 2900, 5)).astype(np.float32)
b = 2
c = 100
t = 5
h = np.transpose(np.reshape(h, [b, c, -1, t]), [0, 1, 3, 2])
h.shape

(2, 100, 5, 29)

In [31]:
unnormalized_widths = h[:, :, :, :num_bins]
unnormalized_heights = h[:, :, :, num_bins:2*num_bins]
unnormalized_derivatives = h[:, :, :, 2 * num_bins:]
unnormalized_widths.shape, unnormalized_heights.shape, unnormalized_derivatives.shape

((2, 100, 5, 10), (2, 100, 5, 10), (2, 100, 5, 9))

In [32]:
inputs_tf = tf.convert_to_tensor(inputs)
inputs_pt = torch.from_numpy(inputs)
unnormalized_widths_pt = torch.from_numpy(unnormalized_widths)
unnormalized_widths_tf = tf.convert_to_tensor(unnormalized_widths)
unnormalized_derivatives_tf = tf.convert_to_tensor(unnormalized_derivatives)
unnormalized_heights_tf = tf.convert_to_tensor(unnormalized_heights)

unnormalized_derivatives_pt = torch.from_numpy(unnormalized_derivatives)
unnormalized_heights_pt = torch.from_numpy(unnormalized_heights)

In [11]:
tail_bound = 1.0
inside_interval_mask = (inputs >= -tail_bound) & (inputs <= tail_bound)
outside_interval_mask = ~inside_interval_mask
inputs_tf[inside_interval_mask].shape

TensorShape([Dimension(706)])

In [12]:
unnormalized_heights_tf.shape

TensorShape([Dimension(2), Dimension(100), Dimension(5), Dimension(10)])

In [13]:
d = tf.shape(unnormalized_widths_tf)[-1]
unnormalized_widths_tf = unnormalized_widths_tf[tf.tile(tf.expand_dims(inside_interval_mask, -1), [1,1,1, d])]
unnormalized_widths_tf = tf.reshape(unnormalized_widths_tf, [-1, d])

d = tf.shape(unnormalized_heights_tf)[-1]
unnormalized_heights_tf = unnormalized_heights_tf[tf.tile(tf.expand_dims(inside_interval_mask, -1), [1,1,1, d])]
unnormalized_heights_tf = tf.reshape(unnormalized_heights_tf, [-1, d])

d = tf.shape(unnormalized_derivatives_tf)[-1]
unnormalized_derivatives_tf = unnormalized_derivatives_tf[tf.tile(tf.expand_dims(inside_interval_mask, -1), [1,1,1, d])]
unnormalized_derivatives_tf = tf.reshape(unnormalized_derivatives_tf, [-1, d])

In [14]:
unnormalized_widths_tf

<tf.Tensor: id=80, shape=(706, 10), dtype=float32, numpy=
array([[-1.9023976 ,  1.0981514 ,  2.0500915 , ...,  0.6847241 ,
         0.7687547 , -0.5549106 ],
       [ 2.05902   ,  1.120718  , -0.48864928, ..., -0.04568215,
        -1.6403083 , -2.2799225 ],
       [ 0.56897897,  0.8485505 ,  1.1065133 , ...,  0.9879283 ,
        -1.2290384 , -0.72810787],
       ...,
       [-2.1979778 ,  1.1859202 ,  0.25839475, ..., -0.08144075,
         1.3474473 , -0.08661917],
       [ 1.7538358 ,  1.2065862 , -0.38038737, ...,  0.90097886,
         0.30276212,  0.18352847],
       [ 0.40946227, -1.3744577 , -1.1465975 , ..., -0.65348977,
        -1.9920306 ,  0.64239913]], dtype=float32)>

In [15]:
def searchsorted(bin_locations, inputs, eps=1e-6):
    bin_locations[..., -1] += eps
    return torch.sum(
        inputs[..., None] >= bin_locations,
        dim=-1
    ) - 1

def searchsorted_tf(bin_locations, inputs, eps=1e-6):
    bin_locations = tf.concat([
        bin_locations[..., :-1],
        tf.expand_dims(bin_locations[..., -1] + 1e-6, -1)], axis=-1)
    return tf.reduce_sum(
        tf.cast(inputs[..., None] >= bin_locations, tf.int32),
        axis=-1
    ) - 1

In [33]:
left=0. 
right=1.
bottom=0.
top=1.
min_bin_width = 1e-3
min_bin_height = 1e-3
min_derivative = 1e-3

In [16]:
unnormalized_widths = unnormalized_widths_tf
num_bins = tf.shape(unnormalized_widths)[-1]

widths = tf.nn.softmax(unnormalized_widths, axis=-1)
widths = min_bin_width + (1 - min_bin_width * tf.cast(num_bins, tf.float32)) * widths
cumwidths = tf.cumsum(widths, axis=-1)
cumwidths = tf.pad(cumwidths, [[0, 0], [1, 0]])
cumwidths = (right - left) * cumwidths + left

In [17]:
unnormalized_derivatives = unnormalized_derivatives_tf
unnormalized_heights = unnormalized_heights_tf

In [18]:
unnormalized_heights.shape

TensorShape([Dimension(706), Dimension(10)])

In [19]:
cumwidths = tf.concat([
    tf.expand_dims(tf.fill(tf.shape(cumwidths[..., 0]), left), -1),
    cumwidths[..., 1:-1],
    tf.expand_dims(tf.fill(tf.shape(cumwidths[..., -1]), right), -1),
], axis = -1)
widths = cumwidths[..., 1:] - cumwidths[..., :-1]

derivatives = min_derivative + tf.math.softplus(unnormalized_derivatives)

heights = tf.nn.softmax(unnormalized_heights, axis=-1)
heights = min_bin_height + (1 - min_bin_height * tf.cast(num_bins, tf.float32)) * heights
cumheights = tf.cumsum(heights, axis=-1)
cumheights = tf.pad(cumheights, [[0, 0], [1, 0]])

# cumheights[..., 0] = bottom
# cumheights[..., -1] = top

cumheights = tf.concat([
    tf.expand_dims(tf.fill(tf.shape(cumheights[..., 0]), bottom), -1),
    cumheights[..., 1:-1],
    tf.expand_dims(tf.fill(tf.shape(cumheights[..., -1]), top), -1),
], axis=-1)

heights = cumheights[..., 1:] - cumheights[..., :-1]

In [20]:
bin_idx_tf = searchsorted_tf(cumwidths, inputs_tf[inside_interval_mask])[..., None]
bin_idx_tf

<tf.Tensor: id=326, shape=(706, 1), dtype=int32, numpy=
array([[-1],
       [ 5],
       [ 4],
       [-1],
       [ 3],
       [ 4],
       [-1],
       [ 1],
       [-1],
       [-1],
       [-1],
       [ 2],
       [-1],
       [-1],
       [-1],
       [-1],
       [ 2],
       [ 5],
       [-1],
       [ 5],
       [ 3],
       [-1],
       [ 6],
       [ 1],
       [ 5],
       [-1],
       [ 2],
       [ 7],
       [ 7],
       [ 4],
       [-1],
       [ 1],
       [ 5],
       [ 1],
       [ 4],
       [-1],
       [ 1],
       [-1],
       [ 0],
       [-1],
       [ 3],
       [-1],
       [-1],
       [ 0],
       [-1],
       [-1],
       [ 0],
       [-1],
       [-1],
       [ 6],
       [-1],
       [-1],
       [ 1],
       [-1],
       [ 8],
       [ 1],
       [-1],
       [ 1],
       [ 9],
       [-1],
       [ 0],
       [-1],
       [ 5],
       [ 7],
       [-1],
       [ 2],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [-1],
       [

In [21]:
bin_idx_tf = tf.where(bin_idx_tf < 0, tf.fill(tf.shape(bin_idx_tf), 0), bin_idx_tf)

In [22]:
derivatives

<tf.Tensor: id=224, shape=(706, 9), dtype=float32, numpy=
array([[0.4595182 , 0.65262556, 1.357251  , ..., 1.1222818 , 0.19808091,
        0.4142052 ],
       [1.0668324 , 0.76401156, 0.41303495, ..., 1.5789281 , 1.3999131 ,
        0.25445667],
       [0.94706815, 0.49312025, 1.2083801 , ..., 0.4050719 , 0.6567668 ,
        0.11223203],
       ...,
       [0.40728635, 1.0551487 , 0.82515085, ..., 1.4267883 , 0.11855583,
        0.7138041 ],
       [0.5961442 , 0.91774267, 0.2459634 , ..., 0.640446  , 1.2767872 ,
        1.3704453 ],
       [0.11431326, 0.26217353, 0.9218413 , ..., 1.1645179 , 1.1166086 ,
        0.07798084]], dtype=float32)>

In [23]:
bin_idx_tf[:,0]

<tf.Tensor: id=336, shape=(706,), dtype=int32, numpy=
array([0, 5, 4, 0, 3, 4, 0, 1, 0, 0, 0, 2, 0, 0, 0, 0, 2, 5, 0, 5, 3, 0,
       6, 1, 5, 0, 2, 7, 7, 4, 0, 1, 5, 1, 4, 0, 1, 0, 0, 0, 3, 0, 0, 0,
       0, 0, 0, 0, 0, 6, 0, 0, 1, 0, 8, 1, 0, 1, 9, 0, 0, 0, 5, 7, 0, 2,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 7, 6, 0, 2, 4, 6, 9, 0, 0, 4, 0, 6,
       0, 5, 0, 0, 2, 0, 1, 0, 9, 1, 0, 6, 8, 0, 9, 7, 0, 3, 3, 5, 5, 6,
       9, 5, 1, 9, 0, 4, 7, 7, 0, 0, 6, 3, 0, 0, 0, 5, 3, 0, 0, 0, 9, 9,
       0, 0, 0, 4, 0, 0, 0, 0, 4, 2, 1, 5, 0, 0, 0, 1, 0, 4, 6, 0, 8, 5,
       0, 9, 0, 5, 7, 2, 8, 4, 5, 9, 0, 8, 0, 2, 0, 5, 3, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 1, 3, 5, 0, 5, 2, 0, 3, 0, 0, 7,
       1, 0, 0, 0, 0, 0, 0, 9, 0, 2, 0, 0, 9, 0, 1, 8, 0, 4, 0, 7, 2, 0,
       7, 3, 0, 7, 1, 0, 2, 0, 0, 1, 8, 8, 0, 0, 0, 0, 0, 6, 5, 4, 0, 0,
       0, 0, 0, 5, 0, 3, 4, 3, 7, 4, 0, 3, 0, 0, 0, 0, 0, 2, 0, 2, 0, 0,
       3, 0, 4, 2, 5, 0, 9, 0, 6, 2, 7, 3, 0, 2, 8, 0, 0, 8, 0, 0, 1, 

In [ ]:
cumwidths_tf = tf.gather_nd(cumwidths, bin_idx_tf, batch_dims = 1)
cumwidths_tf

In [ ]:
tf.gather_nd(derivatives, bin_idx_tf, batch_dims=1)

In [ ]:
derivatives.shape, cumwidths.shape

In [ ]:
np.round(input_cumwidths.numpy(), 3)

In [ ]:
np.round(input_cumwidths.numpy(), 3) == np.round(cumwidths_tf.numpy(), 3)

In [34]:
inside_interval_mask = (inputs_pt >= -tail_bound) & (inputs_pt <= tail_bound)
outside_interval_mask = ~inside_interval_mask
inputs_pt[inside_interval_mask].shape

torch.Size([706])

In [35]:
inside_interval_mask.shape

torch.Size([2, 100, 5])

In [36]:
inputs_pt[inside_interval_mask].shape, unnormalized_widths_pt[inside_interval_mask, :].shape

(torch.Size([706]), torch.Size([706, 10]))

In [37]:
unnormalized_widths = unnormalized_widths_pt[inside_interval_mask, :]
unnormalized_heights=unnormalized_heights_pt[inside_interval_mask, :]
unnormalized_derivatives=unnormalized_derivatives_pt[inside_interval_mask, :]
num_bins = unnormalized_widths.shape[-1]
widths = F.softmax(unnormalized_widths, dim=-1)
widths = min_bin_width + (1 - min_bin_width * num_bins) * widths
cumwidths = torch.cumsum(widths, dim=-1)
cumwidths = F.pad(cumwidths, pad=(1, 0), mode='constant', value=0.0)
cumwidths = (right - left) * cumwidths + left
cumwidths[..., 0] = left
cumwidths[..., -1] = right

In [38]:
widths = cumwidths[..., 1:] - cumwidths[..., :-1]

derivatives = min_derivative + F.softplus(unnormalized_derivatives)

heights = F.softmax(unnormalized_heights, dim=-1)
heights = min_bin_height + (1 - min_bin_height * num_bins) * heights
cumheights = torch.cumsum(heights, dim=-1)
cumheights = F.pad(cumheights, pad=(1, 0), mode='constant', value=0.0)
cumheights = (top - bottom) * cumheights + bottom
cumheights[..., 0] = bottom
cumheights[..., -1] = top
heights = cumheights[..., 1:] - cumheights[..., :-1]

In [40]:
def searchsorted(bin_locations, inputs, eps=1e-6):
    bin_locations[..., -1] += eps
    return torch.sum(
        inputs[..., None] >= bin_locations,
        dim=-1
    ) - 1

In [41]:
bin_idx = searchsorted(cumwidths, inputs_pt[inside_interval_mask])[..., None]
bin_idx[bin_idx < 0] = 0
bin_idx.shape

torch.Size([706, 1])

In [42]:
input_cumwidths = cumwidths.gather(-1, bin_idx)[..., 0]

In [47]:
cumheights.shape, cumwidths.shape

(torch.Size([706, 11]), torch.Size([706, 11]))

In [45]:
cumheights.gather(-1, bin_idx)[..., 0].shape

torch.Size([706])

In [48]:
 derivatives.gather(-1, bin_idx)[..., 0]

RuntimeError: index 9 is out of bounds for dimension 1 with size 9

In [ ]:
inputs_pt[inside_interval_mask].shape, cumwidths.shape

In [ ]:
torch.sum(bin_idx, dim=-1) - 1

In [ ]:
unnormalized_widths_pt[inside_interval_mask, :]

In [ ]:
cumwidths = unnormalized_widths_pt[inside_interval_mask, :]

In [ ]:
cumwidths.gather(-1, bin_idx)[..., 0]

In [ ]:
input_cumwidths = cumwidths.gather(-1, bin_idx)[..., 0]
input_bin_widths = widths.gather(-1, bin_idx)[..., 0]

In [ ]:
idx = searchsorted_tf(inputs_tf[inside_interval_mask], unnormalized_widths_tf)
idx

In [ ]:
unnormalized_widths_pt[inside_interval_mask, :]

In [ ]:
tf.broadcast_to()

In [ ]:
unnormalized_widths = torch.randn(2, 100, 5, 2)

In [ ]:
unnormalized_derivatives = F.pad(unnormalized_derivatives, pad=(1, 0))
unnormalized_derivatives.shape

In [ ]:
inputs = torch.randn(2, 100, 5)

In [ ]:
tail_bound = 1.0
inside_interval_mask = (inputs >= -tail_bound) & (inputs <= tail_bound)
outside_interval_mask = ~inside_interval_mask

In [ ]:
inputs[inside_interval_mask].shape

In [ ]:
unnormalized_widths[inside_interval_mask, :].shape

In [ ]:
outputs = tf.zeros_like(inputs)

In [ ]:
inputs.shape[-1]

In [ ]:
tf.where(outside_interval_mask, outputs, inputs)

In [ ]:
outside_interval_mask

In [ ]:
x = torch.randn(2, 100, 5)
inputs = torch.randn(2, 100)

inputs[..., None] >= x

In [ ]:
x = tf.random.normal(shape = (2, 5, 100))
x = tf.expand_dims(x, 2)
x = depthwise_conv(x)[:,:,0]

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [ ]:
%%time

sess.run(x)

In [49]:
logs_p_np = np.random.normal(size = (2, 50, 5))
m_p_np = np.random.normal(size = (2, 50, 5))
z_p_np = np.random.normal(size = (2, 50, 100))

logs_p = torch.from_numpy(logs_p_np)
z_p = torch.from_numpy(z_p_np)
m_p = torch.from_numpy(m_p_np)

In [50]:
s_p_sq_r = torch.exp(-2 * logs_p) # [b, d, t]
neg_cent1 = torch.sum(-0.5 * math.log(2 * math.pi) - logs_p, [1], keepdim=True) # [b, 1, t_s]
neg_cent2 = torch.matmul(-0.5 * (z_p ** 2).transpose(1, 2), s_p_sq_r) # [b, t_t, d] x [b, d, t_s] = [b, t_t, t_s]
neg_cent3 = torch.matmul(z_p.transpose(1, 2), (m_p * s_p_sq_r)) # [b, t_t, d] x [b, d, t_s] = [b, t_t, t_s]
neg_cent4 = torch.sum(-0.5 * (m_p ** 2) * s_p_sq_r, [1], keepdim=True) # [b, 1, t_s]
neg_cent = neg_cent1 + neg_cent2 + neg_cent3 + neg_cent4

In [51]:
neg_cent.shape

torch.Size([2, 100, 5])

In [52]:
neg_cent

tensor([[[ -797.5473,  -302.9780,  -418.0751,  -142.3627,  -334.3927],
         [ -271.6225,  -166.9982,  -447.6349,  -257.3925,  -319.8517],
         [ -624.2477,  -260.5945,  -758.0973,  -538.4803,  -393.2479],
         [ -201.4374,  -287.3746,  -443.6294,  -192.3147,  -313.4371],
         [ -438.0231,  -371.3127,  -455.8598,  -223.4654,  -255.6165],
         [ -238.7374,  -255.0612,  -305.8648,  -201.8587,  -283.6645],
         [ -373.0448,  -178.7985,  -369.9525,  -235.0557,  -258.8540],
         [ -138.4327,  -241.8564,  -850.4899,  -253.1263,  -494.4948],
         [ -165.6693,  -283.8421,  -301.3271,  -218.5451,  -311.8951],
         [ -172.7845,  -344.0888,  -280.8932,  -395.5952,  -435.7254],
         [ -521.7657,  -147.4768,  -245.2557,  -267.9266,  -297.3464],
         [ -454.4290,  -362.2886,  -376.1942,  -193.9801,  -303.2535],
         [ -125.7292,  -228.7482,  -411.4695,  -241.6433,  -365.3022],
         [ -324.2196,  -262.7355,  -241.9627,  -201.5056,  -437.7278],
      

In [53]:
logs_p = tf.transpose(tf.convert_to_tensor(logs_p_np), [0, 2, 1])
z_p = tf.transpose(tf.convert_to_tensor(z_p_np), [0, 2, 1])
m_p = tf.transpose(tf.convert_to_tensor(m_p_np), [0, 2, 1])

In [54]:
logs_p.shape

TensorShape([Dimension(2), Dimension(5), Dimension(50)])

In [55]:
s_p_sq_r = tf.exp(-2 * logs_p)
neg_cent1 = tf.reduce_sum(-0.5 * np.log(2 * np.pi) - logs_p, axis=-1)[:, None]
neg_cent2 = tf.matmul(-0.5 * (z_p ** 2), tf.transpose(s_p_sq_r, [0, 2, 1]))
neg_cent3 = tf.matmul(z_p, tf.transpose((m_p * s_p_sq_r), [0, 2, 1]))
neg_cent4 = tf.reduce_sum(-0.5 * (m_p ** 2) * s_p_sq_r, axis=-1)[:, None]
neg_cent_tf = neg_cent1 + neg_cent2 + neg_cent3 + neg_cent4

In [56]:
np.round(neg_cent_tf.numpy(), 3) == np.round(neg_cent.numpy(), 3)

array([[[ True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True],
        [ 